In [ ]:
import pandas as pd
import numpy as np

def evaluate_provider_combinations(df):

    # Ensure columns exist
    required_cols = [
        "providertaxid", "providernpi", 
        "sel_category", "paid_amount_bucket",
        "audits", "findings", "nofindings",
        "op_amount", "dispute_count", "overturn_count"
    ]

    for col in required_cols:
        if col not in df.columns:
            df[col] = 0   # missing columns default to 0 so algorithm won't break

    # -------------------------------------------------------
    # Step 1: Derived metrics (dynamic)
    # -------------------------------------------------------
    df["hitrate"] = df["findings"] / df["audits"].replace(0, np.nan)
    df["dispute_rate"] = df["dispute_count"] / df["audits"].replace(0, np.nan)
    df["overturn_rate"] = df["overturn_count"] / df["findings"].replace(0, np.nan)

    # -------------------------------------------------------
    # Step 2: Dynamic scoring within each provider
    # -------------------------------------------------------
    def score_per_provider(group):

        metric_cols = ["hitrate", "op_amount", "audits"]

        # Compute z-scores only if variation exists
        for col in metric_cols:
            if group[col].nunique() > 1:
                group[col + "_z"] = (group[col] - group[col].mean()) / group[col].std()
            else:
                group[col + "_z"] = 0

        # Weighted final score (can adjust weights anytime)
        group["score"] = (
            0.5 * group["hitrate_z"] +
            0.3 * group["op_amount_z"] +
            0.2 * group["audits_z"]
        )

        return group

    df = df.groupby(["providertaxid", "providernpi"], group_keys=False).apply(score_per_provider)

    # -------------------------------------------------------
    # Step 3: Classification into buckets
    # -------------------------------------------------------
    def classify(row):
        if row["audits"] < 15:
            return "Insufficient Data – low audit sample"
        if row["score"] > 0.7:
            return "High Performing"
        if 0.4 <= row["score"] <= 0.7:
            return "Moderate Performing"
        if row["score"] < 0.4:
            return "Low Performing"
    
    df["performance_label"] = df.apply(classify, axis=1)

    # -------------------------------------------------------
    # Step 4: Insight Generation
    # -------------------------------------------------------
    def create_insight(row):

        if row["performance_label"].startswith("Insufficient"):
            return (
                f"Only {row['audits']} audits — not enough data to judge performance "
                f"in {row['sel_category']} – {row['paid_amount_bucket']}."
            )

        if row["performance_label"] == "High Performing":
            return (
                f"Strong performance in {row['sel_category']} – {row['paid_amount_bucket']}: "
                f"Hitrate {row['hitrate']:.2%}, OP ${row['op_amount']:.2f}. "
                f"Among the best categories for this provider."
            )

        if row["performance_label"] == "Moderate Performing":
            return (
                f"Average performance in {row['sel_category']} – {row['paid_amount_bucket']}. "
                f"Hitrate and OP are close to provider's overall average."
            )

        if row["performance_label"] == "Low Performing":
            return (
                f"Poor performance in {row['sel_category']} – {row['paid_amount_bucket']}: "
                f"Hitrate {row['hitrate']:.2%}, OP ${row['op_amount']:.2f}. "
                f"Much lower than other categories for this provider. "
                f"Recommended for deprioritization/exclusion."
            )

    df["insight"] = df.apply(create_insight, axis=1)

    return df


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

def evaluate_provider_performance(df):
    # Compute basic metrics
    df['hitrate'] = df['findings'] / df['audits']
    df['op_per_audit'] = df['overpayment'] / df['audits']
    df['dispute_rate'] = df['disputes'] / df['audits']
    df['overturn_rate'] = df['overturns'] / df['audits']

    # Remove combinations with <15 audits ONLY FROM EVALUATION
    df['eligible'] = df['audits'] >= 15

    # Standardize metrics provider-wise (TIN + NPI level)
    def provider_zscores(x):
        metrics = ['hitrate','op_per_audit','overturn_rate']
        for m in metrics:
            x[f'{m}_z'] = zscore(x[m], ddof=1) if x[m].nunique() > 1 else 0
        return x

    df = df.groupby(['providertaxid','providernpi']).apply(provider_zscores)

    # Overall performance score (higher = better)
    df['performance_score'] = (
        df['hitrate_z'] * 0.5 +
        df['op_per_audit_z'] * 0.3 +
        df['overturn_rate_z'] * 0.2
    )

    # Insights (dynamic – provider-wise quartiles)
    def assign_insight(x):
        if len(x[x['eligible']]) == 0:
            x['insight'] = "Not enough audit volume to evaluate"
            return x

        scores = x.loc[x['eligible'], 'performance_score']
        q1, q3 = scores.quantile([0.25, 0.75])

        def label(row):
            if not row['eligible']:
                return "Low audits — excluded from evaluation"

            if row['performance_score'] >= q3:
                return ("Strong performer — This selection category & paid bucket "
                        "consistently shows high hitrate, OP and overturn success "
                        "compared to other combinations for this provider")

            elif row['performance_score'] <= q1:
                return ("Weak performer — This combination underperforms in hitrate, OP "
                        "and overturn rate compared to other categories for the same provider")

            else:
                return ("Moderate performer — Performance is neither high nor low but "
                        "should be monitored relative to other categories")

        x['insight'] = x.apply(label, axis=1)
        return x

    df = df.groupby(['providertaxid','providernpi']).apply(assign_insight)

    return df
